In [11]:
import numpy as np #importing all the necessary library
from sklearn.decomposition import PCA
import re
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import KMeansSMOTE
from sklearn.decomposition import TruncatedSVD
from scipy import sparse
from scipy. sparse import csr_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt 
from imblearn.over_sampling import ADASYN 
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import BernoulliNB

In [12]:
training_datafile = "trainingdata.txt" #storing the training and test data files
testing_datafile = "testingdata.txt"

In [13]:
def dataloading(name_of_file, type_of_file): #function is defined to load the dataset
    with open(name_of_file, "r") as read_file:#reading the dataset
        nums = read_file.readlines()

    if type_of_file == "trainingdata":
        Training_labels = [int(l[0]) for l in nums] #converting to sparse data
        for index, item in enumerate(Training_labels):
            if (item == 0): 
                Training_labels[index] = -1
        dfile = [re.sub(r'[^\w]', ' ',l[1:]).split() for l in nums]

    else:
        Training_labels = []
        dfile = [re.sub(r'[^\w]', ' ',l).split() for l in nums]

    Training_features = []

    for f in dfile:
      num = [0]*100001 #creating the list with size of the feature as 100001
      for index, h in enumerate(f):
        num[int(h)] = 1
      Training_features.append(num)

    return Training_features, Training_labels

In [14]:
Training_features, Training_labels = dataloading(training_datafile, "trainingdata") #loading the training data


In [15]:
Truncated_SVD = TruncatedSVD(algorithm='randomized', n_components=500, n_iter=51, random_state=42) #using SVD for dimensionality reduction

Truncated_SVDfitting = Truncated_SVD.fit(Training_features, Training_labels)
reduced_Training_features = Truncated_SVDfitting.transform(Training_features)#the further reduced dimensions are stored




In [16]:
sm_oversampler = SMOTE(random_state=42)#using smote to randomly increase the minority class which here is 1
reduced_Training_features,Training_labels = sm_oversampler.fit_resample(reduced_Training_features,Training_labels)


In [17]:
Testing_features, Testing_labels = dataloading(testing_datafile, "test")#loading the test dataset


Testing_featuresReduced = Truncated_SVDfitting.transform(Testing_features)#reducing the features of test dataset


In [18]:
nameofclassifier = ["Decision Tree","Bernoulli Naive Bayes"]#defining the classifiers
classifier_def =[DecisionTreeClassifier(random_state=42),BernoulliNB()]





In [27]:
for name, val in zip(nameofclassifier, classifier_def):#using the classifier
    print(name)
    
    crossval_calculation = cross_val_predict(val, reduced_Training_features, Training_labels, cv=10)#used for calculation of cross validation 
    print (metrics.classification_report(Training_labels, crossval_calculation))

    tot_score = cross_val_score(val, reduced_Training_features, Training_labels)
    
    print ('\nCross validation scores after performing the classification: ')
    print (tot_score.mean())

   
    val.fit(reduced_Training_features, Training_labels)#the training dataset values are being trained

    
    predicted_results = val.predict(Testing_featuresReduced)#values are predicted for test dataset

    

    FinalOutputFile = 'format.txt' #final output is stored

    print ('storing the output in', FinalOutputFile)

    finaloutput = open(FinalOutputFile, 'w')#opening the file to write
    for i in predicted_results:
        if int(i) == -1:
            i = 0
        
        finaloutput.write(str(i))
        
        finaloutput.write("\n")
    
    finaloutput.close() #closing the file

Decision Tree
              precision    recall  f1-score   support

          -1       0.94      0.89      0.91       722
           1       0.90      0.94      0.92       722

    accuracy                           0.92      1444
   macro avg       0.92      0.92      0.92      1444
weighted avg       0.92      0.92      0.92      1444


Cross validation scores after performing the classification: 
0.9009491541714725
storing the output in format.txt
Bernoulli Naive Bayes
              precision    recall  f1-score   support

          -1       0.86      0.98      0.91       722
           1       0.98      0.84      0.90       722

    accuracy                           0.91      1444
   macro avg       0.92      0.91      0.91      1444
weighted avg       0.92      0.91      0.91      1444


Cross validation scores after performing the classification: 
0.9058270857362551
storing the output in format.txt
